In [2]:
import json
import random

random.seed(0)

def generate_sample(max_digits,
                    num_summands_max,
                    num_summands_min,
                    with_sol=True,
                    with_rationale=True,
                    tool_use=False,
                    exclusion_set=None,
                    operators=['+']):
    while True:
        list_of_digits = [random.randint(0, 10**max_digits - 1) for _ in range(random.randint(num_summands_min, num_summands_max))]
        str_query_core = f'{list_of_digits[0]}'
        for digit in list_of_digits[1:]:
            str_query_core += f' {random.choice(operators)} {digit}'
        int_sol = eval(str_query_core)
        str_query = f'Question: {str_query_core} = ? Answer:'
        if str_query not in exclusion_set:
            break
    list_query_core = str_query_core.split()
    stmts = []
    while len(list_query_core) > 1:
        partial_result = eval(' '.join(list_query_core[:3]))
        if tool_use:
            stmts.append(' '.join(list_query_core[:3] + ['=']))
        else:
            stmts.append(' '.join(list_query_core[:3] + ['=', str(partial_result)]))
        list_query_core = [str(partial_result)] + list_query_core[3:]
    assert int(list_query_core[0]) == int_sol
    str_rationale = ', '.join(stmts)
    str_sol = f'. The answer is {int_sol}.'
    res = {'str_query': str_query, 'num_sol': int_sol}
    if with_rationale:
        res['str_rationale'] = str_rationale
    if with_sol:
        res['str_sol'] = str_sol
    return res

num_samples_train = 50
num_samples_hw = 250
num_samples_test = 500

operators = ['+', '-']
max_digit_len = 1

for num_summands in [3]:
    exclusion_set = set()

    data_json = []
    for _ in range(num_samples_train):
        data_json.append(generate_sample(max_digit_len,
                                         num_summands_max=4,
                                         num_summands_min=3,
                                         with_rationale=True,
                                         with_sol=True,
                                         tool_use=True,
                                         exclusion_set=exclusion_set,
                                         operators=operators))
        exclusion_set.add(data_json[-1]['str_query'])
    with open(f'data/arithmetic_with_tool/{max_digit_len}digit_34_op{"".join(operators)}_train_{num_samples_train}.json', 'w') as f:
        json.dump(data_json, f, indent=4)

    data_json = []
    for _ in range(num_samples_hw):
        data_json.append(generate_sample(max_digit_len,
                                         num_summands_max=4,
                                         num_summands_min=3,
                                         with_rationale=False,
                                         with_sol=True,
                                         exclusion_set=exclusion_set,
                                         operators=['+', '-']))
        exclusion_set.add(data_json[-1]['str_query'])
    with open(f'data/arithmetic_with_tool/{max_digit_len}digit_34_op{"".join(operators)}_hw.json', 'w') as f:
        json.dump(data_json, f, indent=4)

    data_json = []
    for _ in range(num_samples_test):
        data_json.append(generate_sample(max_digit_len,
                                         num_summands_max=5,
                                         num_summands_min=5,
                                         with_rationale=False,
                                         with_sol=False,
                                         exclusion_set=exclusion_set,
                                         operators=['+', '-']))
    with open(f'data/arithmetic_with_tool/{max_digit_len}digit_5_op{"".join(operators)}_test.json', 'w') as f:
        json.dump(data_json, f, indent=4)